In [ ]:
import pandas as pd
import numpy as np

#countvectorizer model
import nltk
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

#tf-idf model
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel



In [ ]:
df = pd.read_csv("../data/goodreads_final_bagowords.csv")
df.head()

## EDA

In [ ]:
df.info()

In [ ]:
df["bookTitle"].nunique()

In [ ]:
df["Author"].nunique()

In [ ]:
df["Genre"].nunique()

## Count Vectorizer

In [ ]:
#CountVectorizer model
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

cosine_sim = cosine_similarity(count_matrix, count_matrix)
print(cosine_sim)

indices = pd.Series(df['bookTitle'])

In [ ]:
test = count_matrix.toarray()
test


In [ ]:
len(test[0])

In [ ]:
sum(test[0])

In [ ]:
def recommend(bookTitle, cosine_sim = cosine_sim):
    recommended_book = []
    idx = indices[indices == bookTitle].index[0]
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_book.append(list(df['bookTitle'])[i])
        
    return recommended_book

In [ ]:
recommend('To Kill a Mockingbird')

## TF- IDF

In [28]:
df.head()

,url,bookTitle,bookImage,bookDesc,bookRating,ratingCount,reviewCount,Genre,pageCount,Author,lang,title_key_words,clean_keywords,sentiment_polarity,sentiment_subjectivity,genre_bag,bag_of_words
0,https://www.goodreads.com/book/show/2767052-th...,The Hunger Games,https://i.gr-assets.com/images/S/compressed.ph...,"Could you survive on your own in the wild, wit...",4.32,6717635,176054,Young Adult,374,Suzanne Collins,en,"['hunger', 'games']","['could', 'survive', 'wild', 'every', 'one', '...",0.101623,0.484921,['youngadult'],youngadult hunger games could survive wild eve...
1,https://www.goodreads.com/book/show/2.Harry_Po...,Harry Potter and the Order of the Phoenix,https://i.gr-assets.com/images/S/compressed.ph...,There is a door at the end of a silent corrido...,4.50,2668409,45724,Fantasy,870,J.K. Rowling,en,"['harry', 'potter', 'order', 'phoenix']","['door', 'end', 'silent', 'corridor', 'hauntin...",-0.087273,0.420909,['fantasy'],fantasy harry potter order phoenix door end si...
2,https://www.goodreads.com/book/show/2657.To_Ki...,To Kill a Mockingbird,https://i.gr-assets.com/images/S/compressed.ph...,The unforgettable novel of a childhood in a sl...,4.28,4772918,95595,Classics,324,Harper Lee,en,"['kill', 'mockingbird']","['unforgettable', 'novel', 'childhood', 'sleep...",0.165686,0.368067,['classics'],classics kill mockingbird unforgettable novel ...
3,https://www.goodreads.com/book/show/1885.Pride...,Pride and Prejudice,https://i.gr-assets.com/images/S/compressed.ph...,Alternate cover edition of ISBN 9780679783268S...,4.27,3206070,74020,Classics,279,Jane Austen,en,"['pride', 'prejudice']","['alternate', 'cover', 'edition', 'isbn', 'sin...",0.475556,0.680000,['classics'],classics pride prejudice alternate cover editi...
4,https://www.goodreads.com/book/show/41865.Twil...,Twilight,https://i.gr-assets.com/images/S/compressed.ph...,About three things I was absolutely positive.F...,3.61,5231000,107619,Young Adult,501,Stephenie Meyer,en,['twilight'],"['three', 'things', 'absolutely', 'positive', ...",0.240000,0.620000,['youngadult'],youngadult twilight three things absolutely po...


In [29]:
#imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
#filter dataframe
rating_min = 3.0

df_fil = df.loc[df['bookRating'] >= rating_min]
df_fil.info()

In [ ]:
#https://practicaldatascience.co.uk/data-science/how-to-create-content-recommendations-using-tf-idf
#bag of words is already made, but this will also down-weight common words that appear across documents
tfidf = TfidfVectorizer(stop_words='english',smooth_idf=True)

# numbers to calculate similarities
tfidf_matrix = tfidf.fit_transform(df_fil['bag_of_words']).todense()

#calculate cosine matrix
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)
print(cosine_similarities)

indices = pd.Series(df_fil['bookTitle'])

In [ ]:
#get recommendations based on tf-idf and cosine_similarities

def recommend_tf(bookTitle, cosine_similarities = cosine_similarities):
    
    recommended_book = []
    recommended_book_url = []
    recommended_book_image= []
    recommended_book_author = []
    recommended_book_rating =[]
    recommended_book_descrip =[]
    recommended_book_genre =[]
    
    idx = indices[indices == bookTitle].index[0]
    
    score_series = pd.Series(cosine_similarities[idx]).sort_values(ascending = False)
    
    top_10_indices = list(score_series.iloc[1:11].index)
    
    for i in top_10_indices:
        recommended_book.append(list(df_fil['bookTitle'])[i])
        recommended_book_url.append(list(df_fil['url'])[i])
        recommended_book_image.append(list(df_fil['bookImage'])[i])
        recommended_book_author.append(list(df_fil['Author'])[i])
        recommended_book_rating.append(list(df_fil['bookRating'])[i])
        recommended_book_descrip.append(list(df_fil['bookDesc'])[i])
        recommended_book_genre.append(list(df_fil['Genre'])[i])
        recommended_book_score.append(score_series[i])

    data = {'Title': recommended_book,
           'Author': recommended_book_author,
            'Genre': recommended_book_genre,
            'Description' : recommended_book_descrip,
            'Rating':recommended_book_rating,
            'URL': recommended_book_url,
            'Image': recommended_book_image,
            'Score': recommended_book_score
           }
    rec_df = pd.DataFrame(data)
    
    return rec_df

In [ ]:
recommend_tf('Twilight')

## Process Model for flask app

In [4]:
import pandas as pd
import numpy as np

In [30]:
#code that stays consistent
df = pd.read_csv("../data/goodreads_final_bagowords.csv")

tfidf = TfidfVectorizer(stop_words='english',smooth_idf=True)
tfidf_matrix = tfidf.fit_transform(df['bag_of_words']).todense()
cosine_similarities = linear_kernel(tfidf_matrix, tfidf_matrix)

#make into dataframe
#df_data = pd.DataFrame(cosine_similarities).reset_index()

#string columns names required for parquet file
#df_data.columns = df_data.columns.astype(str)
#df_data.info()

/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


In [31]:
#cosine_similarities is a numpy.ndarray
type(cosine_similarities)

numpy.ndarray

In [32]:
np.save("../data/goodreads_cos_data.npy", cosine_similarities)

In [33]:
arr1 = np.load("../data/goodreads_cos_data.npy")

In [34]:
print(arr1[0:5])

[[1.         0.01135253 0.         ... 0.         0.         0.        ]
 [0.01135253 1.         0.         ... 0.013395   0.01680991 0.        ]
 [0.         0.         1.         ... 0.0059668  0.         0.        ]
 [0.         0.         0.01156535 ... 0.02099148 0.         0.00368417]
 [0.03437746 0.01256453 0.01654681 ... 0.0131886  0.00470204 0.        ]]


In [9]:

link= 'https://book-rec-nlp.s3.amazonaws.com/goodreads_cos_data.npy'
arr1 = np.load(link)
print(arr1[0:5])

FileNotFoundError: [Errno 2] No such file or directory: 'https://book-rec-nlp.s3.amazonaws.com/goodreads_cos_data.npy'

In [10]:
import s3fs

In [13]:
s3 = s3fs.S3FileSystem(anon=True)

In [14]:
arr1= np.load(s3.open("book-rec-nlp/goodreads_cos_data.npy"))

In [15]:
print(arr1[0:5])

[[1.         0.00402374 0.0138527  ... 0.00703557 0.00161726 0.01776498]
 [0.00402374 1.         0.         ... 0.03376514 0.01499929 0.01502661]
 [0.0138527  0.         1.         ... 0.         0.01281669 0.01778946]
 [0.00876162 0.         0.03632484 ... 0.         0.00672774 0.02887838]
 [0.02501998 0.0097621  0.02640575 ... 0.         0.00894217 0.01068593]]


In [24]:
import boto3
import io
from botocore import UNSIGNED
from botocore.client import Config

In [25]:
s3client = boto3.client('s3', region_name='us-east-1', config=Config(signature_version=UNSIGNED))


In [26]:

with io.BytesIO(s3client.get_object(Bucket="book-rec-nlp", Key="goodreads_cos_data.npy")["Body"].read()) as f:
    f.seek(0)  # rewind the file
    arr1 = np.load(f)

In [27]:
print(arr1[0:5])

[[1.         0.00402374 0.0138527  ... 0.00703557 0.00161726 0.01776498]
 [0.00402374 1.         0.         ... 0.03376514 0.01499929 0.01502661]
 [0.0138527  0.         1.         ... 0.         0.01281669 0.01778946]
 [0.00876162 0.         0.03632484 ... 0.         0.00672774 0.02887838]
 [0.02501998 0.0097621  0.02640575 ... 0.         0.00894217 0.01068593]]


In [ ]:


#variable
bookTitle = 'To Kill a Mockingbird'


#variable reliant code
indices = pd.Series(df_par['bookTitle'])

recommended_book = []
recommended_book_url = []
recommended_book_image= []
recommended_book_author = []
recommended_book_rating =[]
recommended_book_descrip =[]
recommended_book_genre =[]
recommended_book_score =[]

idx = indices[indices == bookTitle].index[0]

score_series = pd.Series(arr1[idx]).sort_values(ascending = False)

top_10_indices = list(score_series.iloc[1:11].index)

for i in top_10_indices:
    recommended_book.append(list(df_par['bookTitle'])[i])
    recommended_book_url.append(list(df_par['url'])[i])
    recommended_book_image.append(list(df_par['bookImage'])[i])
    recommended_book_author.append(list(df_par['Author'])[i])
    recommended_book_rating.append(list(df_par['bookRating'])[i])
    recommended_book_descrip.append(list(df_par['bookDesc'])[i])
    recommended_book_genre.append(list(df_par['Genre'])[i])
    recommended_book_score.append(score_series[i])

data = {'Title': recommended_book,
       'Author': recommended_book_author,
        'Genre': recommended_book_genre,
        'Description' : recommended_book_descrip,
        'Rating':recommended_book_rating,
        'URL': recommended_book_url,
        'Image': recommended_book_image,
        'Score': recommended_book_score
       }
rec_df = pd.DataFrame(data)
rec_df

In [ ]:
#explore variables
idx

In [ ]:
score_series

In [ ]:
top_10_indices

In [ ]:
df_data.index

### Explore data output

In [ ]:
data["Title"][0]

In [ ]:
data["Title"][1]